In [1]:
# Engagement Model Template

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations

# 1 -- Load Data

In [3]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [4]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [5]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [6]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [7]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [8]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [9]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_game_correct
ros_game_incorrect
ros_game_start
ros_games_session
ros_in_game
ros_mistake_made
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [10]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


### Filter Feature Set (Optional)

In [11]:
# features_to_keep = data[0].columns

# always include basic_cols, add desired group of features 
features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [12]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [30]:
for n in range(1, len(part)):
    comb = combinations(part, n)
    for c in comb:
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)
        
        

Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
Train: [9] 	Test: [5, 7, 11, 12, 17, 18]
Train: [11] 	Test: [5, 7, 9, 12, 17, 18]
Train: [12] 	Test: [5, 7, 9, 11, 17, 18]
Train: [17] 	Test: [5, 7, 9, 11, 12, 18]
Train: [18] 	Test: [5, 7, 9, 11, 12, 17]
Train: [5, 7] 	Test: [9, 11, 12, 17, 18]
Train: [5, 9] 	Test: [7, 11, 12, 17, 18]
Train: [5, 11] 	Test: [7, 9, 12, 17, 18]
Train: [5, 12] 	Test: [7, 9, 11, 17, 18]
Train: [5, 17] 	Test: [7, 9, 11, 12, 18]
Train: [5, 18] 	Test: [7, 9, 11, 12, 17]
Train: [7, 9] 	Test: [5, 11, 12, 17, 18]
Train: [7, 11] 	Test: [5, 9, 12, 17, 18]
Train: [7, 12] 	Test: [5, 9, 11, 17, 18]
Train: [7, 17] 	Test: [5, 9, 11, 12, 18]
Train: [7, 18] 	Test: [5, 9, 11, 12, 17]
Train: [9, 11] 	Test: [5, 7, 12, 17, 18]
Train: [9, 12] 	Test: [5, 7, 11, 17, 18]
Train: [9, 17] 	Test: [5, 7, 11, 12, 18]
Train: [9, 18] 	Test: [5, 7, 11, 12, 17]
Train: [11, 12] 	Test: [5, 7, 9, 17, 18]
Train: [11, 17] 	Test: [5, 7, 9, 12, 18]
Train: [11, 18] 

In [48]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_train, split_test

In [49]:
# Function: Formulate Train-Test Split 

# train_part: an array of participants 
# test_part: single participant
# split_size: how much of test_part data to use for testing
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i])
    
    i = part.index(test_part)
    test_data = data[i]
    if split_size < 1:
        split_train, test_data = part_split(test_data, split_size)
        train_data.append(split_train)
    
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    
    return train_data, test_data

In [50]:
# Train-Test Split


# Individualized
# perc = 0.7
perc = 1
# train, test = split(train_part=[7, 9, 11, 12, 17, 18], test_part=5, split_size=perc)
# train, test = split(train_part=[5, 9, 11, 12, 17, 18], test_part=7, split_size=perc)
# train, test = split(train_part=[5, 7, 11, 12, 17, 18], test_part=9, split_size=perc)
# train, test = split(train_part=[5, 7, 9, 12, 17, 18], test_part=11, split_size=perc)
# train, test = split(train_part=[5, 7, 9, 11, 17, 18], test_part=12, split_size=perc)
train, test = split(train_part=[5, 7, 9, 11, 12, 18], test_part=17, split_size=perc)
# train, test = split(train_part=[5, 7, 9, 11, 12, 17], test_part=18, split_size=perc)

### Divide into X_train, y_train, X_test, y_test

In [52]:
y_train = train['engagement']
X_train = train.drop(columns=basic_cols)

y_test = test['engagement']
X_test = test.drop(columns=basic_cols)

In [53]:
# Sort Training Data Together 

X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

# 5 -- Preprocessing
Standardization Recommended

### Min-Max Scaling
- x' = ( x - min(x) ) / ( max(x) - min(x) )
- => x' in [0, 1], fill NaN with -1

In [54]:
# for c in X_train.columns:
#     min_val = np.nanmin(X_train[c])
#     max_val = np.nanmax(X_train[c])
    
#     X_train[c] = (X_train[c]-min_val)/(max_val - min_val)
#     X_test[c] = (X_test[c]-min_val)/(max_val - min_val)
    
# X_train = X_train.fillna(-1)
# X_test = X_test.fillna(-1)

### Standardization (Recommended)
- x' = ( x - mean(x) ) / ( stdev(x) )
- => x' is z-score, NaN's filled with min_val

In [55]:
for c in X_train.columns:
    mean = np.nanmean(X_train[c])
    std = np.nanstd(X_train[c])
    
    X_train[c] = (X_train[c]-mean)/(std)
    X_test[c] = (X_test[c]-mean)/(std)
        
    if 'of_pose_distance' not in c:
        min_val = np.nanmin(X_train[c])

        X_train[c] = X_train[c].fillna(min_val)
        X_test[c] = X_test[c].fillna(min_val)
    else:
        max_val = np.nanmax(X_train[c])

        X_train[c] = X_train[c].fillna(max_val)
        X_test[c] = X_test[c].fillna(max_val)

# 6 -- Model!

In [56]:
success = max(X_test['of_success'].unique())

In [57]:
pred = []
for i in X_test['of_success']:
    if i==success:
        pred.append(1)
    else:
        pred.append(0)

In [58]:
scores = pred

# 7 -- Evaluation

In [59]:
# y_test: actual y values for test data
# scores: predicted probabilistic y values for test data
# pred: predicted binary y values for test data

In [60]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied")
    
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC undefinied
Accuracy: 0.6285178236397748
[[   0    0   48]
 [   0 2578 4052]
 [   0  652 5462]]
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        48
         0.0       0.80      0.39      0.52      6630
         1.0       0.57      0.89      0.70      6114

   micro avg       0.63      0.63      0.63     12792
   macro avg       0.46      0.43      0.41     12792
weighted avg       0.69      0.63      0.60     12792



/anaconda3/envs/caispp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,a_harmonicity,a_harmonicity_var,a_intensity,a_intensity_var,a_mfcc_0,a_mfcc_0_var,a_mfcc_1,a_mfcc_1_var,a_pitch_frequency,a_pitch_frequency_var,...,ros_skill_OS,ros_skill_OS_change,ros_ts_attempt,ros_ts_attempt_var,ros_ts_game_start,ros_ts_game_start_var,ros_ts_robot_talked,ros_ts_robot_talked_var,session_num,timestamp
11629,-200.0000,0.000000,38.4575,2.593553,464.2960,1.391795e+03,323.2740,288.258566,962.7545,7.552455e+04,...,1.0,0,13.983335,0.083241,13.983335,0.083241,2.016665,0.083241,5.0,1802.0
11630,-200.0000,0.000000,38.6080,2.696461,483.4365,1.656075e+03,327.5470,244.025844,938.1890,4.649025e+04,...,1.0,0,14.483335,0.083241,14.483335,0.083241,2.516665,0.083241,5.0,1802.5
11631,-200.0000,0.000000,38.4620,2.302917,516.6345,2.857480e+03,329.5990,284.337551,899.2755,7.434656e+04,...,1.0,0,14.983335,0.083241,14.983335,0.083241,3.016665,0.083241,5.0,1803.0
11632,-200.0000,0.000000,38.8110,0.862486,553.0920,1.218794e+03,343.2010,578.625628,860.7155,6.746361e+04,...,1.0,0,15.483335,0.083241,15.483335,0.083241,3.516665,0.083241,5.0,1803.5
11633,-200.0000,0.000000,39.0060,1.602128,516.8115,8.964145e+02,348.1330,292.525674,955.3250,3.696710e+04,...,1.0,0,15.983335,0.083241,15.983335,0.083241,4.016665,0.083241,5.0,1804.0
11634,-200.0000,11681.967774,41.7035,345.064285,606.4020,1.830060e+05,356.1835,17886.405758,1011.7540,3.747776e+03,...,1.0,0,16.483335,0.083241,16.483335,0.083241,4.516665,0.083241,5.0,1804.5
11635,25.1300,4569.727168,76.4475,119.221349,1277.3700,6.842038e+04,586.6365,7260.875832,1031.8465,2.570126e+04,...,1.0,0,16.983335,0.083241,16.983335,0.083241,5.016665,0.083241,5.0,1805.0
11636,19.6645,10617.967778,73.7170,206.420331,1203.4670,2.433462e+06,560.8055,30417.621321,588.6045,1.030197e+05,...,1.0,0,17.483335,0.083241,17.483335,0.083241,5.516665,0.083241,5.0,1805.5
11637,-97.0670,11763.634197,66.4670,150.470834,1075.1970,2.307928e+06,502.4765,27166.065323,369.8345,7.697894e+04,...,1.0,0,17.983335,0.083241,17.983335,0.083241,6.016665,0.083241,5.0,1806.0
11638,-200.0000,11095.363151,60.8465,222.410856,761.0180,1.082901e+05,502.4765,9417.016950,734.9445,8.145249e+04,...,1.0,0,18.483335,0.083241,18.483335,0.083241,6.516665,0.083241,5.0,1806.5
